In [2]:
import xml.etree.ElementTree as ET
import dicom
import matplotlib.pyplot as plt
import glob
import pickle
import numpy as np

In [9]:
SERIES_NN = []
OVERALL_DICT_NN = {}

folder1 = glob.glob("157/*.xml")
folder2 = glob.glob("185/*.xml")
folder3 = glob.glob("186/*.xml")
folder4 = glob.glob("187/*.xml")
folder5 = glob.glob("188/*.xml")
folder6 = glob.glob("189/*.xml")
xml_files = folder1+folder2+folder3+folder4+folder5+folder6 
for file in xml_files:
    tree = ET.parse(file)
    root = tree.getroot()
    
    SERIES_NN.append(root[0][8].text)
    ALL_CORD_NN = []
    image_names_NN = []
    dictionary_NN = {}
    z_position = []
    flg = 0
    for child in root: 

        if child.tag == '{http://www.nih.gov}readingSession': 
            
            for gc in child:
                if gc.tag == '{http://www.nih.gov}nonNodule':  
                    coord_NN = []
                    for gggc in gc:
                        if gggc.tag == '{http://www.nih.gov}imageZposition':
                            z = gggc.text
                            if z in z_position:
                                flg = 0 
                                break
                            else:
                                flg = 1
                                z_position.append(z)
                                
                        if gggc.tag == '{http://www.nih.gov}imageSOP_UID' :
                            if flg == 1:
                                image_names_NN.append(gggc.text)
            
                        if gggc.tag == '{http://www.nih.gov}locus' :
                            if flg == 1:
                                coord_NN.append((int(gggc[0].text),int(gggc[1].text)))
                        
                    if len(coord_NN)!=0:
                        ALL_CORD_NN.append(coord_NN)


    for i in range(0,len(ALL_CORD_NN)): 
        if image_names_NN[i] in dictionary_NN.keys():
            dictionary_NN[image_names_NN[i]].append(ALL_CORD_NN[i])
        else:
            dictionary_NN[image_names_NN[i]] = [ALL_CORD_NN[i]]
    if root[0][8].text in OVERALL_DICT_NN.keys():

        for k in dictionary_NN:
            OVERALL_DICT_NN[root[0][8].text][k] = dictionary_NN[k]
    else:
        OVERALL_DICT_NN[root[0][8].text] = dictionary_NN

In [5]:
with open('OVERALL_DICT_NN.pkl','wb') as odN:
    pickle.dump(OVERALL_DICT_NN, odN)
with open('SERIES_NN.pkl','wb') as sN:
    pickle.dump(SERIES_NN, sN)